In [1]:
from __future__ import print_function
import os
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np

from local_config import CURRENT_DIR
sys.path.append(os.path.join(CURRENT_DIR, ".."))

try:
    from config import Config
    from data_loader import WaymoStixelDataset
except:
    print("failed to load module")
    
from albumentations import (
    Resize,
    Compose,
    CLAHE,
    HueSaturationValue,
    RandomBrightness,
    RandomContrast,
    RandomGamma,
    ToFloat,
    Normalize,
    GaussNoise,
    RandomShadow,
    RandomRain,
)

import utility
import importlib
from models.stixel_net import build_stixel_net_inceptionV3, build_stixel_net_small, build_stixel_net

# TensorFlow
import tensorflow as tf
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping,
)

In [2]:
dt_config = Config()
dt_config.display()


Configurations:
CURRENT_DIR                    /Users/menzweil/Development/StixelNet/obstacle_detection_stixelnet
DATA_PATH                      /Users/menzweil/Development/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo
GROUND_TRUTH_PATH              /Users/menzweil/Development/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo/waymo_train.txt
NUM_EPOCHS                     50
SAVED_MODELS_PATH              saved_models




In [3]:
## Train and Val set


train_aug = Compose(
    [
        GaussNoise(p=1.0),
        RandomShadow(p=0.5),
        RandomRain(p=0.5, rain_type="drizzle"),
        RandomContrast(limit=0.2, p=0.5),
        RandomGamma(gamma_limit=(80, 120), p=0.5),
        RandomBrightness(limit=0.2, p=0.5),
        HueSaturationValue(
            hue_shift_limit=5, sat_shift_limit=20, val_shift_limit=10, p=0.5
        ),
        CLAHE(p=0.5, clip_limit=2.0),
        Normalize(p=1.0),
    ]
)
    
# AUGMENTATIONS DISABLED
train_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_train.txt"),
        batch_size=8,
        transform=None,
        customized_transform=utility.HorizontalFlip(p=0.5),
    )




val_aug = Compose([Normalize(p=1.0)])
val_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_val.txt"),
        transform=None,
    )


/Users/menzweil/Development/miniforge3/envs/StixelNet/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:1826: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/Users/menzweil/Development/miniforge3/envs/StixelNet/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [4]:
def custom_loss(y_actual,y_pred):
 
    mask = tf.cast(tf.math.less(y_actual, tf.constant([1000000000.0])), dtype=tf.float32)    
    custom_loss=tf.reduce_mean(tf.math.abs( tf.math.multiply( (y_actual-y_pred), mask)), axis=-1)  
    return custom_loss

In [5]:
# StixelNet regression setup

#model = build_stixel_net_inceptionV3()
model = build_stixel_net()
opt = optimizers.Adam()
#lossF = losses.MeanSquaredError()

model.compile(loss=custom_loss, optimizer=opt)
model.summary()

2022-03-12 01:08:16.273366: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-12 01:08:16.273498: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1280, 1920, 3)]   0         
_________________________________________________________________
resizing (Resizing)          (None, 320, 480, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 320, 480, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 320, 480, 3)       0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 8, 13, 1536)       54336736  
_________________________________________________________________
flatten (Flatten)            (None, 159744)            0         
__________________________________________________

In [6]:
X,y = train_set[0]
print(np.shape(X))
print(np.shape(y))

(8, 1280, 1920, 3)
(8, 240)


In [7]:
# Training

num_epoch = 1000

callbacks = [
    ModelCheckpoint(
        os.path.join(dt_config.SAVED_MODELS_PATH, "model-{epoch:03d}-{loss:.4f}.h5"),
        monitor="val_loss",
        verbose=1,
        save_best_only=False,
        mode="auto",
        save_freq="epoch",
    ),    
    EarlyStopping(
        monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto"
    ),
]


history = model.fit(
        train_set,       
        validation_data=val_set,      
        epochs=num_epoch,
        callbacks=callbacks,
        shuffle=True,
    )

2022-03-12 01:08:20.900696: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-12 01:08:20.900840: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/1000


2022-03-12 01:08:23.236827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


624/624 [==============================] - ETA: 0s - loss: 605.1008

2022-03-12 01:17:20.750817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


624/624 [==============================] - 641s 1s/step - loss: 605.1008 - val_loss: 560.1996

Epoch 00001: saving model to saved_models/model-001-605.1008.h5
Epoch 2/1000
624/624 [==============================] - 634s 1s/step - loss: 514.2439 - val_loss: 461.0501

Epoch 00002: saving model to saved_models/model-002-514.2439.h5
Epoch 3/1000
624/624 [==============================] - 634s 1s/step - loss: 345.8168 - val_loss: 226.7565

Epoch 00003: saving model to saved_models/model-003-345.8168.h5
Epoch 4/1000
624/624 [==============================] - 634s 1s/step - loss: 173.1335 - val_loss: 105.8344

Epoch 00004: saving model to saved_models/model-004-173.1335.h5
Epoch 5/1000
624/624 [==============================] - 636s 1s/step - loss: 83.2120 - val_loss: 97.8424

Epoch 00005: saving model to saved_models/model-005-83.2120.h5
Epoch 6/1000
624/624 [==============================] - 634s 1s/step - loss: 75.7095 - val_loss: 103.6241

Epoch 00006: saving model to saved_models/model-0